In [14]:
# Tratamiento de datos
import numpy as np
import pandas as pd

# Modelado y evaluación
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import GridSearchCV
import pickle

# Configuración warnings
import warnings
warnings.filterwarnings('once')

In [2]:
df_registrados = pd.read_pickle('../../../datos_finales/archivo_registrados_est_enc2.pkl')
df_registrados.head(2)

,estacion,año,mes,dia_semana,dias_laborables,tiempo,temperatura,humedad,velocidad_viento,usuarios_registrados
0,1.0,1.0,1.0,1.05,1.0,2.01,-0.486274,0.855582,-0.208909,654
1,1.0,1.0,1.0,1.00,1.3,2.01,-0.425690,0.332722,0.687550,670


In [3]:
X = df_registrados.drop("usuarios_registrados", axis = 1)
y = df_registrados["usuarios_registrados"]

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [5]:
param = {"max_depth": [4,6,8], # tiene 19 de profundidad y como hay overfitting lo reducimos a la mitad.
        "max_features": [1,2,3], # como la raíz cuadrada es 3, llegamos hasta 3
        # hemos tenido en cuenta el nº de filas que tenemos en nuestro conjunto de datos para los siguientes.
        "min_samples_split": [10,25,50],
        "min_samples_leaf": [10,25,50]} 

In [6]:
gs_rf = GridSearchCV(
            estimator=RandomForestRegressor(), # tipo de modelo que queremos hacer
            param_grid= param, # que hiperparámetros queremos que testee
            cv=10, # crossvalidation que aprendimos en la lección de regresión lineal intro. 
            return_train_score = True, # para que nos devuelva el valor de las métricas de set de datos de entrenamiento
            scoring="neg_mean_squared_error") # la métrica que queremos que nos devuelva

In [7]:
gs_rf.fit(x_train, y_train)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [4, 6, 8], 'max_features': [1, 2, 3],
                         'min_samples_leaf': [10, 25, 50],
                         'min_samples_split': [10, 25, 50]},
             return_train_score=True, scoring='neg_mean_squared_error')

In [8]:
bosque = gs_rf.best_estimator_
bosque

RandomForestRegressor(max_depth=8, max_features=3, min_samples_leaf=10,
                      min_samples_split=10)

In [9]:
y_pred_test_rf = bosque.predict(x_test)
y_pred_train_rf = bosque.predict(x_train)

In [10]:
def metricas(y_test, y_train, y_test_pred, y_train_pred, tipo_modelo):
    """
    Esta función nos dice las métricas de nuestro modelo.
    Args:
        y_test (Serie): el test de nuestra variable respuesta.
        y_train (Serie): el train de nuestra variable respuesta.
        y_test_pred (Serie): la predicción para el test de nuestra variable respuesta.
        y_train_pred (Serie): la predicción para el train de nuestra variable respuesta.
        tipo_modelo (str): nombre que le damos al modelo.
    Returns:
        dataframe con todas las métricas de nuestro modelo.
    """
    
    resultados = {'MAE': [mean_absolute_error(y_test, y_test_pred), mean_absolute_error(y_train, y_train_pred)],
                'MSE': [mean_squared_error(y_test, y_test_pred), mean_squared_error(y_train, y_train_pred)],
                'RMSE': [np.sqrt(mean_squared_error(y_test, y_test_pred)), np.sqrt(mean_squared_error(y_train, y_train_pred))],
                'R2':  [r2_score(y_test, y_test_pred), r2_score(y_train, y_train_pred)],
                 "set": ["test", "train"]}
    df = pd.DataFrame(resultados)
    df["modelo"] = tipo_modelo
    return df

In [11]:
dt_results = metricas(y_test, y_train, y_pred_test_rf, y_pred_train_rf, "Random Forest")
dt_results

,MAE,MSE,RMSE,R2,set,modelo
0,489.197186,371006.653591,609.103155,0.828124,test,Random Forest
1,494.489409,430317.817289,655.986141,0.827395,train,Random Forest


In [12]:
dt_results.to_pickle('../../../datos_finales/predicciones/metricas_registrados.pkl')

Este modelo devuelve buenas métricas para R2, aunque se observa ligeramente overfitting. Es el mejor modelo para usuarios registrados y será con el que nos quedemos. 

In [13]:

# vamos a crearnos un dataframe, igual que hicimos en la clase anterior con la importancia de cada una de las variables incluidas en el modelo

importancia_predictores = pd.DataFrame(
                            {'predictor': x_train.columns,
                             'importancia': bosque.feature_importances_}
                            )


# ordenamos de mayor a menor los resultados
importancia_predictores.sort_values(by=["importancia"], ascending=False, inplace = True)

# printeamos los resultados
print("Importancia de los predictores en el modelo")
print("-------------------------------------------")
importancia_predictores

Importancia de los predictores en el modelo
-------------------------------------------


,predictor,importancia
1,año,0.349806
6,temperatura,0.232846
0,estacion,0.151448
2,mes,0.111314
3,dia_semana,0.054943
7,humedad,0.049144
8,velocidad_viento,0.034376
5,tiempo,0.013431
4,dias_laborables,0.002692


In [15]:
with open ("../../../datos_finales/predicciones/mejor_modelo_registrados.pkl", "wb") as f:
    pickle.dump(bosque, f)